In [1]:
# !pip install selenium

In [2]:
# !pip install webdriver-manager

In [3]:
# !pip install openpyxl

In [3]:
import selenium
selenium.__version__

'4.11.2'

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
import random

options = webdriver.ChromeOptions()
# 창 띄우지 않기
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)
driver.set_window_size(width=1024, height=990)

# 큐레이터 추천 소장품 검색 페이지
base_url = f'https://www.museum.go.kr/site/main/relic/recommend/list'
# driver(chrome) open
driver.get(base_url)
# 로딩
time.sleep(2)

# 마지막 페이지 얻어오기
last_page = driver.find_element(By.CLASS_NAME, 'allPage').text.split(' ')[3]
last_page = int(last_page)
print("마지막 페이지 : ", last_page)

data = []
for n in range(1, last_page+1) :
    page_url = f'https://www.museum.go.kr/site/main/relic/recommend/list?cp={n}'
    # 탭 생성
    driver.execute_script("window.open('');")
    # 가장 끝 탭으로 이동
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(random.randint(1, 3))
    # 링크 이동
    driver.get(page_url)
    time.sleep(random.randint(2, 6))

    links = driver.find_elements(By.CLASS_NAME, 'k-cura-txt > a')
    titles = driver.find_elements(By.CSS_SELECTOR, '#card1 > li > div > a')
    co_er = driver.find_elements(By.CSS_SELECTOR, '#card1 > li > div > ul > li:nth-child(1) > p')
    si_no = driver.find_elements(By.CSS_SELECTOR, '#card1 > li > div > ul > li:nth-child(2) > p')

    for i in range(len(titles)) :
        link = links[i].get_attribute('href')
        # print("콘텐츠 링크 : ", link)
        driver.execute_script("window.open('');")
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(random.randint(2, 5))
        driver.get(link)
        time.sleep(3)
        
        # BeautifulSoup로 html parser 이용
        soup = bs(driver.page_source, 'html.parser')

        # 소제목과 내용을 따로 받아오기 -> 소제목이 없는 문단도 있어서 일단 보류
        # sub_titles = soup.select('.prg > h5')
        # paragraphs = soup.select('.prg > p')
        # content_titles = [sub_title.get_text(strip=True) for sub_title in sub_titles]
        # content_paragraphs = [paragraph.get_text(strip=True) for paragraph in paragraphs]
        # if not content_titles:
        #     content_text = content_paragraphs
        # else :
        #     content_text = [f"{title}: {paragraph}" for title, paragraph in zip(content_titles, content_paragraphs)]

        # h5-소제목, p-본문
        paragraphs = soup.select('.prg > h5, .prg > p')
        # paragraphs를 텍스트 리스트로 저장
        content_text = [paragraph.get_text(strip=True) for paragraph in paragraphs]
        # 이걸 리스트로 받아오는 게 맞나? 생각해보기..
        # 제대로 나오는지 확인
        print("콘텐츠 내용 : ", content_text)

        driver.close()
        driver.switch_to.window(driver.window_handles[-1])
        data.append({
            'link' : link,
            'title' : titles[i].text,
            'country_era' : co_er[i].text,
            'size_no' : si_no[i].text,
            'content' : content_text
        })
        time.sleep(2)
    driver.close()
    driver.switch_to.window(driver.window_handles[-1])
    # break

df = pd.DataFrame(columns=['link', 'title', 'country_era', 'size_no', 'content'], data=data)
df

마지막 페이지 :  31
콘텐츠 내용 :  ['성벽 속에서 나온 두 점의 초두', '초두(鐎斗, 자루솥)는 고대에 만든 세 개의 다리와 긴 손잡이가 달린 금속 냄비입니다. 세 다리 사이에 불을 피워 술이나 차 또는 약을 끓였다고 합니다. 초두는 주로 청동이나 철로 만들었지만 드물게 자기로 된 것도 있습니다. 중국 한나라 때 처음 만들었으며, 우리나라에서는 삼국시대부터 사용하기 시작했습니다.1925년 한반도 전역에 7월부터 9월까지 큰비가 내렸습니다. 이때 한강도 넘쳐서 많은 논밭과 민가가 불어난 물에 잠기고 말았습니다. 특히 동부 이촌동·뚝섬·송파·잠실리·신천리·풍납리 등 한강과 가깝거나 낮은 지대가 큰 피해를 보았습니다. 이를 두고 을축년 대홍수라고 부릅니다. 이 재해로 수많은 사상자와 조선총독부 1년 예산의 58%에 이르는 약 1억 300만 원의 피해액이 발생했습니다.그런데 홍수로 허물어진 경기도 광주군 풍납리의 옛 성벽(오늘날의 서울 풍납토성) 속에서 큰 항아리 하나가 드러났습니다. 그 항아리에는 청동 거울과 두 점의 청동 초두가 들어 있었습니다. 국립중앙박물관이 소장한 유리건판 가운데 당시 큰물이 지나간 뒤에 찍은 성벽과 항아리 사진이 있습니다.', '비슷하지만 서로 다른 모양', '두 청동 초두의 몸체는 높이가 낮고 바닥이 납작한 원통 모양이고, 아가리는 바깥으로 짧게 벌어져 있습니다. 다리와 손잡이를 몸통에 붙이기 전에 덧판을 먼저 대고 다리와 손잡이를 붙을 부분을 미리 뚫어둔 것도 공통점입니다. 하지만 찬찬히 살펴보면 다른 점이 많습니다. 손잡이를 기준으로 삼으면 긴 손잡이가 달린 초두[靑銅製長柄鐎斗]와 용머리 손잡이가 달린 초두[靑銅製龍首柄鐎斗]로 구분할 수 있습니다. 긴 손잡이가 달린 초두의 모양을 먼저 살피면, 손잡이가 몸체보다 길게 수평으로 뻗어 있고 손잡이 끝은 스페이드(Spade) 꼴입니다. 아가리에는 액체를 따를 때 필요한 긴 귀때가 붙어 있습니다. 세 다리에는 중간 지점보다 아래쪽에 대나무 마디처럼 뾰족하게 돋은 부분이 있습니다. 다

,link,title,country_era,size_no,content
0,https://www.museum.go.kr/site/main/relic/recom...,"청동 초두, 무덤이 아닌 성벽 속에서 발견되다","삼국시대 (백제, 4~5세기)","입지름 14.6cm, 16.5cm\n본관9954, 본관9955","[성벽 속에서 나온 두 점의 초두, 초두(鐎斗, 자루솥)는 고대에 만든 세 개의 다..."
1,https://www.museum.go.kr/site/main/relic/recom...,《삼현수간첩(三賢手簡帖)》 - 세 선비의 마음을 담은 편지,"조선 (1560~1593년 작성, 1599년 편집)",38.5*27.5cm\n보물\n건희 52,"[《삼현수간첩(三賢手簡帖)》은 1599년(선조 32) 구봉(龜峯) 송익필(宋翼弼, ..."
2,https://www.museum.go.kr/site/main/relic/recom...,『광해군일기』 - 묘호(廟號)를 받지 못한 임금의 역사,조선 (1634년),44.5*31.0cm\n국보\n구 4760,[『광해군일기(光海君日記)』는 조선의 역사를 기록한 『조선왕조실록(朝鮮王朝實錄)』(...
3,https://www.museum.go.kr/site/main/relic/recom...,백제 무령왕릉에서 출토된 받침 있는 은잔[銅托銀盞],백제 (529년 이전),"전체 높이 12.1cm, 전체 너비 14.7cm\n공주 672",[1971년 우연한 계기로 발견된 백제 무령왕릉(武寧王陵)은 충청남도 공주시에 있습...
4,https://www.museum.go.kr/site/main/relic/recom...,따뜻한 위로를 건네는 불화 - <감로도>,조선 (1649년),220*235cm\n신수 2743,"[이 불화는 <감로도(甘露圖)>, 달콤한 이슬이 내리는 장면을 그린 그림입니다. 그..."
...,...,...,...,...,...
298,https://www.museum.go.kr/site/main/relic/recom...,"묵소거사자찬(黙笑居士自讚), 김정희",한국(韓國)-조선(朝鮮) 1840년 이전,32.7x136.4cm\n보물(寶物)\n본관(本館) 005112 00000,[힘든 상황에 처했을 때 나를 이해해 줄 사람이 그리워집니다. 그 대상이 가족일 수...
299,https://www.museum.go.kr/site/main/relic/recom...,"매화초옥도(梅花草屋圖), 전기",한국(韓國)-조선(朝鮮) 19세기 중엽,"32.4x36.1cm(전체), 29.4x33.3cm(그림)\n동원(東垣) 00331...",[‘매화서옥(梅花書屋)’ 또는 ‘매화초옥(梅花草屋)’이라는 제목의 그림은 흔히 둘레...
300,https://www.museum.go.kr/site/main/relic/recom...,"임한경명첩(臨漢鏡銘帖), 김정희",조선시대 19세기,33.8×26.7 cm\n덕수(德壽) 900 0,[《임한경명첩(臨漢鏡銘帖)》은 언뜻 보기에는 너무나 멀어 보이는 위의 조합들의 연결...
301,https://www.museum.go.kr/site/main/relic/recom...,"어미닭과 병아리(母鷄領子圖), 변상벽",한국(韓國)-조선(朝鮮) 18세기,101.0×50.0cm\n덕수(德壽) 001810 00000,"[<어미닭과 병아리>는 조선후기 화원화가(畫員畫家) 변상벽(卞相璧, 생몰년 미상)의..."


In [7]:
df.to_csv('../../../Project/실험.csv', encoding='utf-8-sig')
df.to_json('../../../Project/실험.json', force_ascii=False, orient='records', indent=4) # orient index로 해도 괜춘

## 정리 코드

In [4]:
df.to_json('25p.json', force_ascii=False, orient='index', indent=4)

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
import random
import json

options = webdriver.ChromeOptions()
# 창 띄우지 않기
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)
driver.set_window_size(width=1024, height=990)

# 큐레이터 추천 소장품 검색 페이지
base_url = f'https://www.museum.go.kr/site/main/relic/recommend/list'
# driver(chrome) open
driver.get(base_url)
# 로딩
time.sleep(2)

# 마지막 페이지 얻어오기
last_page = driver.find_element(By.CLASS_NAME, 'allPage').text.split(' ')[3]
last_page = int(last_page)
print("마지막 페이지 : ", last_page)

data = []
failed_get_urls = []

# 1페이지부터 얻어오려면 range 범위 1부터로 조정하기
for n in tqdm(range(25, last_page+1), desc='Crawling Progress..') :
    page_url = f'https://www.museum.go.kr/site/main/relic/recommend/list?cp={n}'
    # 탭 생성
    driver.execute_script("window.open('');")
    # 가장 끝 탭으로 이동
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(random.randint(1, 3))
    # 링크 이동
    driver.get(page_url)
    time.sleep(random.randint(2, 6))
    
    try :
        soup = bs(driver.page_source, "html.parser")
        cards = soup.find_all("li", class_="card")
        for card in cards :
            tag_a = card.find("a", class_="img-box")
            title = card.find('div',class_='k-cura-txt').find('a').get_text()
            country_and_era = card.find('strong', string='국적/시대').find_next_sibling('p').get_text(strip=True)
            size_and_category = card.find('strong', string='크기/지정구분').find_next_sibling('p').get_text(separator='\n', strip=True)
            href = tag_a['href']
            link = f"https://www.museum.go.kr{href}"
            # print("콘텐츠 링크 : ", link)
            driver.execute_script("window.open('');")
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(random.randint(2, 5))
            try :
                driver.get(link)
                time.sleep(3)
                
                # BeautifulSoup로 html parser 이용
                soup = bs(driver.page_source, 'html.parser')

                paragraphs = soup.select('.prg > h5, .prg > p')
                # paragraphs를 텍스트 리스트로 저장
                content_text = [paragraph.get_text(strip=True) for paragraph in paragraphs]
            except Exception as e_content :
                print(f'An error occurred at content {title} : {e_content}')
                # content_text = '콘텐츠 저장 실패' # 이거 쓸까 말까..

            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
            data.append({
                'title' : title,
                'country_and_era' : country_and_era,
                'size_and_category' : size_and_category,
                'content' : content_text
            })
            time.sleep(2)
    except Exception as e:
        print(f"An error occurred at page {n}: {e}")
        failed_get_urls.append(page_url)
        
    driver.close()
    driver.switch_to.window(driver.window_handles[-1])

if failed_get_urls :
    df_file = pd.DataFrame(failed_get_urls).to_csv('fail_url.csv', encoding='utf-8-sig')
df = pd.DataFrame(columns=['title', 'country_and_era', 'size_and_category', 'content'], data=data)
time.sleep(2)

df.to_csv('25p.csv', encoding='utf-8-sig')
df.to_json('25p.json', force_ascii=False, orient='index', indent=4)

마지막 페이지 :  31


Crawling Progress..: 100%|██████████| 7/7 [13:08<00:00, 112.69s/it]


In [9]:
len_test = pd.read_csv('25p.csv')
len(len_test['content'][2])

3719